# Redes neuronales hacia adelante

En esta libreta vamos a realizar las funciones necesarias para entrenar y predecir utilizando uno red neuronal hacia adelante multicapa, con función de activación logística en *todas* las neuronas de las capas ocultas. Esta libreta no pretende sustituir a las explicaciones en clase o a unas notas sobre redes neuronales. Aqui se asume que ustedes ya tienen una idea general de las redes neuronales, que comprenden las ecuaciones de aprendizaje como las obtuvimos en clase, así como los algoritmos básicos. En esta libreta *solamente* nos vamos a centrar en los aspectos de implementación.

Para empezar esta libreta, necesitaremos algunas de las funciones que ya programamos en libretas pasadas, las cuales adjuntamos a continuación.

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cPickle
from scipy.optimize import minimize
from IPython.display import Image 


def obtiene_medias_desviaciones(x):
    """
    Obtiene las medias y las desviaciones estandar atributo a atributo.
    
    @param x: un ndarray de dimensión (T, n) donde T es el númro de elementos y n el número de atributos
    @return: medias, desviaciones donde ambos son ndarrays de dimensiones (n,) con las medias y las desviaciones 
             estandar respectivamente.
    
    """
    return x.mean(axis=0), x.std(axis=0)

def normaliza(x, medias, desviaciones):
    """
    Normaliza los datos x

    @param x: un ndarray de dimensión (T, n) donde T es el númro de elementos y n el número de atributos
    @param medias: ndarray de dimensiones (n,) con las medias con las que se normalizará
    @param desviaciones: ndarray de dimensiones (n,) con las desviaciones con las que se normalizará
    
    @return: x_norm un ndarray de las mismas dimensiones de x pero normalizado
    
    """
    return (x - medias) / desviaciones


def logistica(z):
    """
    Calcula la función logística para cada elemento de z
    
    @param z: un ndarray
    @return: un ndarray de las mismas dimensiones que z
    """
    return 1 / (1 + np.exp(-z))

def softmax(z):
    """
    Calculo de la regresión softmax
    
    @param z: ndarray de dimensión (T, K) donde z[i, :] es el vector de aportes lineales de el objeto i    
    @return: un ndarray de dimensión (T, K) donde cada columna es el calculo softmax de su respectivo vector de entrada.
    
    """
    y_hat = np.exp(z)
    return y_hat / y_hat.sum(axis=1).reshape(-1,1)


## 1. Especificando una red neuronal

Primero, para poder hacer una red neuronal, tenemos que determinar cierta información. Por el momento, y para efecto de la libreta, vamos a mantenernos en un esquema estructurado/funcional, y más adelante vamos a generalizar la mayoría de los métodos utilizados en forma de objetos.

La información importante que debemos de tener en cuenta cuando hacemos un sistema de redes neuronales es:

- Cuantas capas de neuronas tiene la red neuronal, $L$.
- Cuantas neuronas va a tener cada capa $[n_0, n_1, \ldots, n_L]$, donde $n_0$ es el número de entradas y $n_L$ el número de salidas.
- Cual es el tipo de salida de mi red neuronal (lineal, logística o softmax)
- Los valores con los que se normalizan los datos de entrada a la red neuronal (para el aprendizaje en una red neuronal es muy importante que los valores de entrada estén normalizados).

Una vez que se establecen estos valores, es necesario generar una lista de matrices $[W^{(1)}, \ldots, W^{(L)}]$ donde $W^{(l)}$ es una matriz de dimensiones $(n_l, n_{l-1} + 1)$ de parámetros o pesos. Como vimos en clase, si se inicializan los valores de las entradas de $W^{(l)}$ iguales, es equivalente a tener una sola neurona en esa capa, por lo que es necesario que estos valores sean diferentes. 

En clase vimos que, para efectos de un mejor aprendizaje, es importante que los valores de entrada se encientren en la zona donde casua más variacion la función logística. Para esto, se espera que en general la suma de los pesos multiplicados por las entradas correspondientes a la capa se encuentren en el rango de $(-1, 1)$. Si asumimos que las entradas a cada neurona están normalizadas (esto es, entre 0 y 1), entonces los pesos deberían ser valores entre $(-\sqrt{n_{l-1}}, \sqrt{n_{l-1}})$ con el fin que la suma se encuentre en la región donde más cambios ocurren en la función logística. 

Vamos a generar y guardar esta información en un diccionario (junto con el resto de la información que requeriramos para tener una red neuronal completamente definida. Al principio los valores de normalización no cuentan ya que estos se deben inicializar al comienzo del aprendizaje.

#### Ejercicio 1. Completa el código de la función de inicialización para los pesos de las matrices de pesos (10 puntos).

In [6]:
def inicializa_red_neuronal(capas, neuronas_por_capa, tipo):
    """
    Inicializa una red neuronal como un diccionario de datos
    
    @param capas: Un número entero con el número total de capas. Minimo 3 (una de entrada, una oculta, una de salida).
    @param neuronas_por_capa: Una lista de enteros donde el primer elemento es el número de entradas
                              y el último el número de salidas, mientras que los intermedios son
                              el númerode neuronas en cada capa oculta.
    @param tipo: Un string entre {'lineal', 'logistica', 'softmax'} con el tipo de salida de la red.
    
    @return: Un diccionario tal que
             - dicc['capas'] = capas
             - dicc['nxc'] = neuronas_por_capas
             - dicc['tipo'] = tipo
             - dicc['thetas'] = lista de matrices de parámetros
             - dicc['medias'] = lista de medias de cada atributo
             - dicc['std'] = lista de desviaciones estandard de cada atributo
             
    """
    if capas != len(neuronas_por_capa):
        raise ValueError('El número de capas no corresponde con la lista de las neuronas por capa')
    dicc = {'capas': L, 'nxc': neuronas_por_capa, 'tipo': tipo}
    dicc['medias'] = np.zeros(neuronas_por_capa[0])
    dicc['std'] = np.ones(neuronas_por_capa[0])
    
    lista_Thetas = []
    for l in range(1, capas + 1):
        lista_thetas.append(inicializa_Theta(dicc['nxc'][l - 1], dicc['nxc'][l]))
    dicc['thetas'] = lista_Thetas
    
    return dicc

def inicializa_Theta( n_lm1, n_l):
    """
    Inicializa una matriz de valores aleatorios Theta
    
    @param n_lm1: número de neuronas en la capa l-1 (entero)
    @param n_l: número de neuronas en la capa l (entero)
    
    @return: Un ndarray de dimensión (n_l, n_lm1 + 1) donde las entradas son número aleatorios
             entre -sqrt(n_lm1) y sqrt(n_lm1)
             
    """
    #------------------------------------------------------------------------
    # Agregua aqui tu código
    return 2.0 * (np.random.random((n_l, n_lm1 + 1)) - 0.5) / np.sqrt(n_lm1)
    #-------------------------------------------------------------------------

def test_inicializa_theta():
    #Vamos a hacer 1000 pruebas aleatorias que nos aseguremos que se cumpleen con las especificaciones
    for _ in range(1000):
        n0 = np.random.randint(1, 20)
        n1 = np.random.randint(1, 20)
        Theta1 = inicializa_Theta( n0, n1)
        assert Theta1.shape == (n1, n0 + 1)  # Las dimensiones son correctas
        assert Theta1.max() < np.sqrt(n0)    # La cota máxima se respeta
        assert Theta1.min() > -np.sqrt(n0)   # La cota mínima se respeta
        assert np.abs(Theta1).sum() > 0      # No estamos inicializando a 0
    return "Paso la prueba"

print test_inicializa_theta()
    

Paso la prueba


Así, si tenemos una red neuronal, la información contenida en el diccionario es toda la información específica que se necesita para la predicción, el aprendizaje, o el reaprendizaje de una red ya especificada. 

Como entrenar una red es algo lento y tedioso, y normalmente cuando hacemos un método de aprendizaje, lo que queremos es poder utilizarlo después para predecir un conjunto de datos no etiquetados previamente, es normal que guardemos en un archivo la información específica a la red neuronal, y despues la recuperemos en otra sesión, otro día, o en otra computadora para hacer la predicción.

Una manera de guardar datos, funciones y objectos de Python en disco es utilizando el módulo ``pickle`` (o su versión compilada para mayor velocidad ``cPickle``). Este modulo permite guardar una serie de objetos de python en forma secuencial en un archivo binario, y luego recuperarlos. Notese que este métdo es diferente a ``np.load``y ``np.savez``, ya que estos solo permiten guardar (y recuperar) una serie de ndarrays únicamente. 

Vamos entonces a hacer dos funciones muy simples ``guarda_objeto`` y ``carga_objeto``, que utilizaremos más adelante.

In [7]:
def guarda_objeto(archivo, objeto):
    """
    Guarda un objeto de python en el archivo "archivo". Si el archivo existe, sera reemplazado sin 
    preguntas, al puro estilo mafioso.
    
    @param archivo: string con el nombre de un archivo (aunque no exista)
    @param objeto: Un objeto de python para ser guardado
    
    """
    
    with open(archivo, 'wb') as arch:
        cPickle.dump(objeto, arch, -1)
        arch.close()
        
def carga_objeto(archivo):
    """
    Carga el primer (y se asume que único) objeto contenido en el archivo 'archivo' que debe de ser tipo cPickle.
    
    @param archivo: string con el nombre de un archivo tipo pickle
    @return: El primer objeto dentro del picke
    
    """
    with open(archivo, 'rb') as arch:
        objeto = cPickle.load(arch)
        arch.close()
        return objeto
    
def test_archivo():
    """
    Prueba, para esto vamos a cargar o a leer (o ambas cosas) un objeto en un archivo
    
    Por favor, borrar el archivo cada vez que se pruebe, o para probar la lectura y la escritura
    
    """
    temp = [range(100), 'prueba', True]
    guarda_objeto('prueba.pkl', temp)
    temp =[10, 'no prueba', False]
    otro = carga_objeto('prueba.pkl')
    assert len(otro[0]) == 100
    assert otro[1] == 'prueba'
    assert otro[-1]
    
    return "Pasa la prueba"

print test_archivo()

Pasa la prueba


## 2. Calculando el costo (y por lo tanto feed-forward).

Asumamos que tenemos una red neuronal ya inicializada, y que la vamos a utilizar para calcular el costo de una solución. Como vimos en clase, el costo de la solución depende del tipo de neuronas de salida (que son en realidad la etapa de clasificación). Así, para calcular el costo, es necesario calcular la salida de la red neuronal.

Recordemos que el algoritmo para realizar la alimentación hacia adelante de una red neuronal el algoritmo es el siguiente:

1. Inicializa $a^{(0)}$ asignandole los valores de las entradas

2. Por cada capa $l$ de 1 a $L-1$:

    1. Se calcula el valor de $z^{(l)}$ como $$z^{(l)} = \Theta^{(l-1)} a_e^{(l-1)},$$ donde $\Theta^{(l-1)}$ es la 
       matriz de pesos de la capa $l-1$ a la capa $l$, y $a_e{(l-1)}$ es $a^{(l-1)}$ extendida con un 1 al principio 
       el vector.
       
    2. Se calcula $a^{(l)}$ como $$a^{(l)} = g(z^{(l)}),$$ donde $g$ es la función de activación (en nuestro caso hemos 
       decidido utilizar la función logística, pero podríamos tener otras funciones de activación).

3. Se calcula el valor de $z^{(L)}$ como $$z^{(L)} = \Theta^{(L-1)} a_e^{(L-1)}.$$ 

4. Se calcula $a^{(L)}$ de acuerdo a la función de activación dependiendo del tipo de salida:

    * Si `tipo = 'logistica'` entonces se utiliza la regresión logística (una sola neurona en la capa de salida).
    * Si `tipo = 'lineal'` entonces $a^{(L)} = z^{(L)}$.
    * Si `tipo = 'softmax'` entonces $a^{(L)} = softmax(z^{(L)}).$

5. La salida de la red es $a^{(L)}$.

Aqui hay que tomar en cuenta varias cosas: en primer lugar, la activación de todas las neuronas en todas las capas, y para todos los datos los necesitamos para realizar el algoritmo de *backpropagation*, por lo que se requiere guardarlos. Igualmente, no es eficiente calcular todos los pasos dato por dato, ya que eso lo haría muy, pero muy lento. Así que vamos a aporvechar que los datos vienen en forma de un *ndarray* de numpy y haremos todos los calculos en forma matricial tal como los vimos en clases. 

Sea $X$ la matriz de valores de entrada, entonces $A^{0} = X^T$ es una lista de vectores columna donde cada columna es $a^{(0)}$ para el objeto correspondiente. Así, los calculos se pueden realizar columna por columna, y simplemente
$$ 
Z^{(l)} = \Theta^{(l-1)}A_e^{(l-1)},
$$
donde $A_e^{(l-1)}$ es $A^{(l-1)}$ agregandole un 1 al inicio de cada vector (o lo que es lo mismo, agregandole un renglon de unos al inicio). Si procedemos de esta forma, entonces es importante recordar que al final $\hat{Y} = (A^{(L)})^T$, ya que para la salida cada renglon es un dato diferente (de acuerdo a nuestra convención desde los otros algoritmos que hamos utilizado), mientras que internamente,para la red neuronal, cada columna proviene de un objeto diferente.

Por último, es importante recordar que la normalización es muy importante para las redes neuronales, especialmente si se utiliza el método de descenso de gradiente, por lo que es importante normalizar los datos antes de que sean utilizados, con la información de normalización que se conoce.

#### Ejercicio 2. Completa el código de la función de *feedforward* para una red neuronal ya establecida (30 puntos).

In [4]:
def feedforward(X, red_neuronal):
    """
    Calcula la salida estimada para los valores de `X` utilizando red_neuronal
    
    @param X: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal``
    
    @return: `Y_est`, `lista_A`, donde `Y_est` es un ndarray de shape (T, k) donde T es el número de objetos y k es 
             el número de salidas (neuronas de salida de la red neuronal).
             
    """
    # Primero hay que normalizar los datos de entrada
    # ----------Agragar código aqui -----------------
    xnorm = normaliza(x, red_neuronal['medias'], red_neuronal['std'])
    
    # Despues es necesario inicializar A^{(0)}
    # ----------Agragar código aqui -----------------
    lista_A = []
    lista_A.append(xnorm.T)
    
    # Despues vamos a hacer lo propio por cada capa hasta antes de la última
    for l in range(1, red_neuronal['capas']):
        # Calcula A_e^{l-1}, Z^{(l)} y por último A^{(L)} 
        # y agrega A^{(L)} a lista_A.        
        # (puede ser todo junto o por partes)
        # ----------Agragar código aqui -----------------
        lista_A.append( logistica(red_neuronal['thetas'][l-1].dot(extendida(lista_A[l-1]))))
                                      
    
def prueba_feedforward():
    """
    Función para validar la función de fedforward (cada paso)

    """
    red_neuronal = inicializa_red_neuronal(2, neuronas_por_capa, tipo)


Y con  la función de feedforward desarrollada, entonces podemos hacer una función para calcular el costo final, la cual depende de las salidas `Y`, de las salidas estimadas por la red neuronal `Y_est`, y del tipo de salida. Igualmente, para agregar la regularización, es necesario un valor `lammbda` de regularización y los parámetros de la red neuronal (`lista_thetas` de la estructura de datos de la red neuronal).

#### Ejercicio 3: Completa el código de la función de costo (10 puntos).

In [ ]:
def costo(Y, Y_est, tipo, lammbda=0, thetas=None):
    """
    Calcula la función de costo de una red neuronal con regularización (por default 0)
    
    @param Y
    
    """

## 3. Calculando el gradiente con el algoritmo de *Backpropagation*

Es este apartado se genera el gradiente utilizando simplemente el algoritmo de backpropagarion y se prueba con una red neuronal ya hecha anteriormente.

## 4. Aprendizaje con descenso de gradiente

En este apartado se realiza el decenso de gradiente, incluidos

1. Inercia
2. Parada temprana

y por último se prueba en un problema sencillo (aunque no tan sencillo pues, pa que tenga interés).

## 5. Aprendizaje por función de optimización

Este apartado es muy rápido, solo se usa para ver como se enrrolla y se desenrrolla un vector de parámetros